In [1]:
import pandas as pd

# Three Most Common Recommended System


- **demographic filtering**:
    - tidak personalized / secara general -> **Top 50 Movies Of The Year**
    - filter berdasarkan fitur demographic seperti genre,durasi,dll.
    - sistem rekomendasi yang simpel
    - Ide : Direkomendasikan berdasarkan apa yang disukai orang secara umum
- **content-based filtering**:
    - rekomendasi barang yang serupa -> **Other movies you like**
    - filter menggunakan fitur yang lebih spesifik
    - Ide : Jika seseorang menonton film X, maka dia akan direkomendasikan film yang serupa dengan X
- **collaborative filtering**
    - Mencocokan orang dengan preferensi serupa -> **Other people also watched**
    - tidak memerlukan filter data apapun, hanya memerlukan kemiripan dengan orang lain

# Simple Demographic Filtering: Filter -> Scoring -> Sort

In [2]:
df = pd.read_csv("data/demographic.csv")
df.head()

,title,genres,runtime,vote_average,vote_count,release_year,Action,Adventure,Animation,Comedy,...,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,Toy Story,Animation; Comedy; Family,81.0,7.7,5415.0,1995,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,Jumanji,Adventure; Fantasy; Family,104.0,6.9,2413.0,1995,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Grumpier Old Men,Romance; Comedy,101.0,6.5,92.0,1995,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
3,Waiting to Exhale,Comedy; Drama; Romance,127.0,6.1,34.0,1995,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
4,Father of the Bride Part II,Comedy,106.0,5.7,173.0,1995,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# df.sort_values("vote_average", ascending=False)

# Step 1: Filter

In [6]:
genre = ["Animation"]
duration = (60, 150)
year = (2000, 2019)
topk = 20

In [8]:
df = df[df.release_year.between(year[0], year[1]) &
        df.runtime.between(duration[0], duration[1]) &
        df[genre].all(axis=1)]

df.head()

,title,genres,runtime,vote_average,vote_count,release_year,Action,Adventure,Animation,Comedy,...,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
3161,The Tigger Movie,Animation; Family,77.0,6.3,146.0,2000,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3354,The Road to El Dorado,Adventure; Animation; Comedy; Family,89.0,7.0,892.0,2000,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3484,Dinosaur,Animation; Family,82.0,6.2,563.0,2000,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3614,Titan A.E.,Animation; Action; Science Fiction; Family; Ad...,94.0,6.3,320.0,2000,1,1,1,0,...,0,0,0,0,0,1,0,0,0,0
3619,Chicken Run,Animation; Comedy; Family,84.0,6.5,1190.0,2000,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0


# Step 2: Scoring

Dikasus ini tinggal pakai vote average sebagai score

# Step 3: Sort

In [10]:
recommendation = df.loc[:, "title":"release_year"]
recommendation = recommendation.sort_values("vote_average", ascending=False).head(topk)
recommendation

,title,genres,runtime,vote_average,vote_count,release_year
25671,Rocks in my Pockets,Comedy; Animation; Drama,88.0,9.4,5.0,2014
41890,Kizumonogatari Part 3: Reiketsu,Animation; Fantasy; Mystery,82.0,9.4,5.0,2017
26636,Lotte from Gadgetville,Adventure; Animation; Comedy; Family,81.0,9.0,4.0,2006
41846,Kizumonogatari Part 2: Nekketsu,Animation; Fantasy; Mystery,68.0,8.9,11.0,2016
35577,"Fuse, Memoirs of the Hunter Girl",Action; Animation; Drama; History,110.0,8.8,4.0,2012
43038,In This Corner of the World,Animation; Drama,128.0,8.7,19.0,2016
40018,Your Name.,Romance; Animation; Drama,106.0,8.5,1030.0,2016
28618,The Life and Mind of Mark DeFriest,Animation; Documentary,92.0,8.5,2.0,2014
41695,The Snow Queen 3: Fire and Ice,Family; Animation; Fantasy,80.0,8.5,2.0,2016
26159,The Littlest Angel,Animation; Family,84.0,8.5,2.0,2011
